In [1]:
import os
import pandas as pd
import subprocess
import glob
import pysam
from tqdm import tqdm
import subprocess
import joblib
import pathlib

In [2]:
input_dir = '/ceph/gale-1/qzeng/AmbData/CellType_Allc/'

In [3]:
ages = ['8wk','9mo','18mo']
use_mc_cts = joblib.load('/home/qzeng/project/aging/metadata/mc_use_cts')
use_m3c_cts = joblib.load('/home/qzeng/project/aging/metadata/m3c_use_cts')

In [4]:
shared_cts = list(set(use_mc_cts) & set(use_m3c_cts))
len(shared_cts)

36

## change names

In [6]:
mreged_cts = [pathlib.Path(path).name.split('.')[0] for path in glob.glob('Total.Balanced.Merged.Allc/*.Both-Gender.CGN-Merge.allc.tsv.gz')]
mreged_cts = list(set(mreged_cts))

In [7]:
old_to_new = {'Endo-NN': 'Endo_NN',
 'L6-IT-CTX-Glut': 'L6_IT_CTX_Glut',
 'SUB-ProS-Glut': 'SUB-ProS_Glut',
 'L23-IT-ENTl-PIR-Glut': 'L23_IT_ENTl-PIR_Glut',
 'NDB-SI-MA-STRv-Lhx8-Gaba': 'NDB-SI-MA-STRv_Lhx8_Gaba',
 'NP-SUB-Glut': 'NP_SUB_Glut',
 'Oligo-NN': 'Oligo_NN',
 'STR-PAL-Chst9-Gaba': 'STR-PAL_Chst9_Gaba',
 'L6bCT-ENT-Glut': 'L6bCT_ENT_Glut',
 'L2-IT-PPP-APr-Glut': 'L2_IT_PPP-APr_Glut',
 'VLMC-NN': 'VLMC_NN',
 'STR-D1-Gaba': 'STR_D1_Gaba',
 'CA3-Glut': 'CA3_Glut',
 'MEA-COAp-Glut': 'MEA-COAp_Glut',
 'L23-IT-PPP-Glut': 'L23_IT_PPP_Glut',
 'L23-IT-ENT-Glut': 'L23_IT_ENT_Glut',
 'Sst-Chodl-Gaba': 'Sst_Chodl_Gaba',
 'Lamp5-Gaba': 'Lamp5_Gaba',
 'CA2-FC-IG-Glut': 'CA2-FC-IG_Glut',
 'Astro-NN': 'Astro_NN',
 'ENTmv-PA-COAp-Glut': 'ENTmv-PA-COAp_Glut',
 'L45-IT-CTX-Glut': 'L45_IT_CTX_Glut',
 'LA-BLA-BMA-PA-Glut': 'LA-BLA-BMA-PA_Glut',
 'CHOR-NN': 'CHOR_NN',
 'PAG-Glut': 'PAG_Glut',
 'MB-MY-Glut-Sero': 'MB-MY_Tph2_Glut-Sero',
 'CA1-ProS-Glut': 'CA1-ProS_Glut',
 'PAG-PCG': 'PAG_PCG',
 'Microglia-NN': 'Microglia_NN',
 'STR-D2-Gaba': 'STR_D2_Gaba',
 'L5-IT-CTX-Glut': 'L5_IT_CTX_Glut',
 'PAG-MRN-Pou3f1-Glut': 'PAG-MRN_Pou3f1_Glut',
 'OT-D3-Folh1-Gaba': 'OT_D3_Folh1_Gaba',
 'DGNBL-Glut': 'DGNBL_Glut',
 'L5-NP-CTX-Glut': 'L5_NP_CTX_Glut',
 'L5-ET-CTX-Glut': 'L5_ET_CTX_Glut',
 'Pvalb-Gaba': 'Pvalb_Gaba',
 'L6-CT-CTX-Glut': 'L6_CT_CTX_Glut',
 'CEA-BST-Gaba': 'CEA-BST_Gaba',
 'Sst-Gaba': 'Sst_Gaba',
 'DG-Glut': 'DG_Glut',
 'L23-IT-CTX-Glut': 'L23_IT_CTX_Glut',
 'STR-Gaba': 'STR_Gaba',
 'VipSncg-Gaba': 'VipSncg_Gaba',
 'OPC-NN': 'OPC_NN',
 'MEA-COA-Glut': 'MEA-COAp_Glut',
 'MEA-BST-Gaba': 'MEA-BST_Gaba'}

In [11]:
all_paths = glob.glob('Total.Balanced.Merged.Allc/*.Both-Gender.CGN-Merge.allc.tsv.gz')

In [10]:
for path in all_paths:
    old_ct = pathlib.Path(path).name.split('.')[0]
    if old_ct in old_to_new.keys():
        new_ct = old_to_new[old_ct]
        new_path = path.replace(old_ct, new_ct)
        command = f"mv {path} {new_path}"
        subprocess.run(command, shell = True)
        

## merge remaining

In [15]:
has_cts = [pathlib.Path(path).name.split('.')[0] for path in glob.glob('Total.Balanced.Merged.Allc/*.Both-Gender.CGN-Merge.allc.tsv.gz')]
has_cts = set(has_cts)

In [16]:
shared_cts = [ct.replace('/','').replace(' ','_') for ct in shared_cts]

In [21]:
remerge_cts = ['MEA-COA_Glut',
 'Astro-TE_NN',
 'STR_D1_Sema5a_Gaba',
 'Vip_Gaba',
 'PAG-PPN_Pax5_Sox21_Gaba',
 'Astro-NT_NN',
 'PB_Evx2_Glut',
 'LDT-PCG-CS_Gata3_Lhx1_Gaba']

In [22]:
groups = [f'{ct}.{age}' for ct in remerge_cts for age in ages ]
len(groups)

24

In [23]:
joblib.dump(groups, 'remain_groups')

['remain_groups']

In [9]:
save_dir = '/home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/Total.Balanced.Merged.Allc'

In [ ]:
for group in tqdm(groups):
    ct, age = group.split('.')
    male_path = f"{input_dir}/M.CellType.Age/{group}/{group}.CGN-Merge.allc.tsv.gz"
    female_path = f"{input_dir}/F.CellType.Age/{group}/{group}.CGN-Merge.allc.tsv.gz"
    
    male_df = pd.read_csv(male_path, sep = '\t', header = None)
    female_df = pd.read_csv(female_path, sep = '\t', header = None)
    
    male_df.index = male_df[0] + '-' + male_df[1].astype(str)
    female_df.index = female_df[0] + '-' + female_df[1].astype(str)
    #shared_cytosine_id = list(set(male_df.index) & set(female_df.index))
    shared_cytosine_id = [index for index in male_df.index if index in female_df.index]
    
    male_df = male_df.loc[shared_cytosine_id]
    female_df = female_df.loc[shared_cytosine_id]
    
    male_total_cov = male_df[5].sum()
    female_total_cov = female_df[5].sum()
    div_by = male_total_cov/female_total_cov

    male_df[4] = (male_df[4] / div_by).astype('int32')
    male_df[5] = (male_df[5] / div_by).astype('int32')
    
    allc_df = female_df.copy()
    allc_df[4] = female_df[4] + male_df[4]
    allc_df[5] = female_df[5] + male_df[5]
    
    allc_df.to_csv(f"{save_dir}/{group_name}.Both-Gender.CGN-Merge.allc.tsv", sep = '\t', 
               index = None, header = None)
    
    subprocess.run(f"bgzip {save_dir}/{group_name}.Both-Gender.CGN-Merge.allc.tsv", 
               shell = True)
    
    subprocess.run(f"allcools tabix-allc {save_dir}/{group_name}.Both-Gender.CGN-Merge.allc.tsv.gz", shell = True)
    
    
    

  0%|                                                                                     | 0/24 [00:00<?, ?it/s]